# Импортируем конфиг

In [1]:
import json

In [2]:
with open('config.json', 'r') as config_file: 
    config = json.load(config_file)

config

{'base_url_gtp': 'http://172.20.10.7:1234/v1',
 'model_gpt': 'hermes-3-llama-3.1-8b',
 'base_url_embeddings': 'http://172.20.10.7:1234/v1/embeddings',
 'model_embeddings': 'nomic-embed-text-v1.5',
 'folder_id': 'b1gmu78t1n7ne841vdb0',
 'API': 'Р°ajeb1g9ieu9janbblt1e',
 'api_key': 'AQVNywwTJ3kzVda69C7D0Y2xq_wO42B8sUDAIVju'}

# Обработаем входные данные:
Сконвертируем все в pdf

In [3]:
from file_message import to_all_pdf

In [4]:
path = 'dataset_input'
dataset_folder ='./dataset/'
conv = False

In [5]:
if conv:
    to_all_pdf(path, dataset_folder)

Загрузим  файлы

In [6]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader

In [7]:
loader = DirectoryLoader( path=dataset_folder, loader_cls=PyPDFLoader)

In [8]:
docs = loader.load()

Рзабьем датасет на сигменты:

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language

In [10]:
splitter = RecursiveCharacterTextSplitter.from_language(
        language=Language.MARKDOWN, 
        chunk_size=1024, 
        chunk_overlap=128
    )

In [11]:
split_data = splitter.split_documents(docs)

# Запустим Embeddings

In [15]:
from localEmbeddings import LocalEmbeddings
from langchain.vectorstores import FAISS

In [14]:
model_embeddings = LocalEmbeddings(
    model=config['model_embeddings'],
    linck=config['base_url_embeddings']
    )

In [17]:
vectorstore = FAISS.from_texts(
    [
        i.page_content.replace("\n", " ")
        for i in split_data
    ], 
    embedding=model_embeddings
    )

Сохраним векторное хранилище

In [19]:
vectorstore.save_local('vectorstores')

Загркзка модели:

In [27]:
vectorstore = FAISS.load_local(
    'vectorstores', 
    model_embeddings,
    allow_dangerous_deserialization=True
    )

In [20]:
retriever = vectorstore.as_retriever()

# Запустим чат

In [29]:
from langchain_openai import OpenAI
from langchain_core.prompts import ChatPromptTemplate

Подключим модель для ответов

In [30]:
model = OpenAI(
    base_url=config['base_url_gtp'],
    model=config['model_gpt'],
    api_key='non'
)

In [31]:
def query(text):
    vect = retriever.invoke(text)
    temple = ChatPromptTemplate([
        ("system", 'Дай краткий ответ на вопрос исходи из {vect}'),
        ("user", '{text}')
    ])
    out = model.invoke(
        temple.invoke({
            'vect':vect,
            'text':text
        })
    )
    return out, vect

In [33]:
out = query(input())
print(out[0])

 

AI: Вся деятельность в Сатурн строится исходя из следующих принципов:
1. Все операции с ПА регистрируют владелец ПА вне зависимости от того, кто и где, и на каких основаниях, физически перемещает ПА.
2. Сатурн отслеживает Местоположение и Владельца в отношении каждой партии ПА; обстоятельства размещения ПА в том или ином месте в Сатурн не регистрируются (будь это доверенность, ответственное хранение, аренда, давальческое производство или что-то иное - эти обстоятельства для Сатурн “не интересны”. Важно “чье”, и “где.”)
3. Сатурн это вторичная учетная система , которая “ссылается” на юридически/финансово значимые документы в первичных системах учета ХС. На каждую регистрируемую операцию или документ, у вас должен быть привычный (далее “бумажный”) документ в вашей перв
